## Promise Loop

循环执行多次Promise，并采集结果

下面是一个原型实现

In [1]:
class PromiseLoop {
  constructor (
    limit = Number.MAX_SAFE_INTEGER
  ) {
    this.limit = limit
    this.finished = false
  }

  finish () {
    this.finished = true
  }

  run (promise) {
    let that = this

    let todoList = []
    todoList.length = this.limit
    todoList.fill(promise)

    let results = []

    return todoList.reduce(function (cur, next) {
      return cur.then(next).then((res) => {
        results.push(res)
        if (that.finished) {
          return Promise.reject('finished')
        } else {
          Promise.resolve(res)
        }
      })
    }, Promise.resolve()).then(function () {
      return new Promise((resolve) => {
        resolve({
          status: 'allFinished',
          results: results
        })
      })
    }).catch((res) => {
      if (res === 'finished') {
        return Promise.resolve({
          status: 'finished',
          results: results
        })
      } else {
        return Promise.resolve({
          status: 'error',
          results: res
        })
      }
    })
  }
}

let get = new PromiseLoop(100)

let todo = () => {
  return new Promise((resolve, reject) => {
    setTimeout(() => {
      let res = Math.random()
      console.log('res', res)
      if (res > 0.7) {
        get.finish()
      }
      resolve(res)
    }, 1000)
  })
}

get.run(todo)

res 0.922001277223002


{ status: 'finished', results: [ 0.922001277223002 ] }